## Importing Libraries

In [1]:
import pathlib
import numpy as np
import os
import cv2
from tensorflow import keras
import tensorflow as tf
from sklearn.metrics import classification_report

## Loading Spectrograms

In [ ]:
image_dir = pathlib.Path("./dataset/Spectrograms/")
label_dict = {'CommonCuckoo': 0, 'CommonKestrel': 1, 'CommonRingedPlover': 2, 'LittleOwl': 3, 'NorthernRaven': 4}

In [4]:
ls=[name for name in os.listdir("./dataset/Spectrograms/")]
gg ={}
for i in ls:
    gg[i]=list(image_dir.glob(f'{i}/*'))

print(gg.keys())

dict_keys(['CommonCuckoo', 'CommonKestrel', 'CommonRingedPlover', 'LittleOwl', 'NorthernRaven'])


In [5]:
x,y=[],[]
for bird,images in gg.items():
    for image in images:
        img =cv2.imread(str(image))
        x.append(img)
        y.append(label_dict[bird])

In [6]:
len(x),len(y)

(1000, 1000)

## Spliting Dataset into train and test

In [7]:
x=np.array(x)
y=np.array(y)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

## Creating CNN for classification

In [8]:
model = keras.Sequential([
keras.layers.Conv2D(input_shape=(480,640,3),filters=32, kernel_size=3, strides=2,activation='relu'), 
keras.layers.BatchNormalization(), 
keras.layers.MaxPooling2D(),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling2D(),
keras.layers.BatchNormalization(),

keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, activation='relu'), 
keras.layers.BatchNormalization(),
keras.layers.MaxPooling2D(),

keras.layers.Conv2D(filters=128, kernel_size=3, strides=1, activation='relu'), 
keras.layers.BatchNormalization(),
keras.layers.BatchNormalization(),
keras.layers.Flatten(),
keras.layers.Dense (256, activation='relu'),
keras.layers.BatchNormalization(),
keras.layers.Dropout(0.4),
keras.layers.Dense (5, activation="softmax"),
])

In [9]:
model.compile(optimizer= 'adam',loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 239, 319, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 239, 319, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 119, 159, 32)     0         
 )                                                               


                                                                 
 batch_normalization_1 (Batc  (None, 119, 159, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 59, 79, 32)       0         
 2D)                                                             
                                                                 
 batch_normalization_2 (Batc  (None, 59, 79, 32)       128       
 hNormalization)                                                 
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 77, 64)        18496     
                                                                 
 batch_normalization_3 (Batc  (None, 57, 77, 64)       256       
 hNormalization)                                                 
                                                                 
 max_pooli

## Training the model

In [10]:
with tf.device('/CPU:0'):
    model.fit(x_train, y_train, epochs=8)

Epoch 1/8


c:\Program Files\Python310\lib\site-packages\keras\backend.py:5582: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


25/25 [==============================] - 39s 1s/step - loss: 1.1708 - accuracy: 0.6363
Epoch 2/8
25/25 [==============================] - 37s 1s/step - loss: 0.7707 - accuracy: 0.7200
Epoch 3/8
25/25 [==============================] - 38s 2s/step - loss: 0.6853 - accuracy: 0.7725
Epoch 4/8
25/25 [==============================] - 37s 1s/step - loss: 0.6009 - accuracy: 0.7862
Epoch 5/8
25/25 [==============================] - 37s 1s/step - loss: 0.5504 - accuracy: 0.7925
Epoch 6/8
25/25 [==============================] - 37s 1s/step - loss: 0.4601 - accuracy: 0.8338
Epoch 7/8
25/25 [==============================] - 37s 1s/step - loss: 0.4059 - accuracy: 0.8562
Epoch 8/8
25/25 [==============================] - 37s 1s/step - loss: 0.3124 - accuracy: 0.8838


## Evaluating the model

In [12]:
with tf.device('/CPU:0'):
    model.evaluate(x_test,y_test)

7/7 [==============================] - 4s 529ms/step - loss: 0.7852 - accuracy: 0.7350


## Saving the model

In [13]:
model.save('model.keras')

## Calculating Classification report

In [14]:
with tf.device('/CPU:0'):
    y_pre=model.predict(x_test)

7/7 [==============================] - 3s 346ms/step


In [15]:
y_pred=[np.argmax(i) for i in y_pre]

In [16]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.88      0.66      0.75        44
           1       0.88      0.82      0.85        51
           2       0.73      0.68      0.70        40
           3       0.42      0.64      0.51        25
           4       0.75      0.82      0.79        40

    accuracy                           0.73       200
   macro avg       0.73      0.72      0.72       200
weighted avg       0.77      0.73      0.74       200

